<h1>贝叶斯机器学习 Bayesian Machine Learning</h1>

参考资料:

[1]<a href="https://metacademy.org/roadmaps/rgrosse/bayesian_machine_learning">https://metacademy.org/roadmaps/rgrosse/bayesian_machine_learning</a>

[2] PRML

# 0. 主要问题

## 0.0 参数估计 Parameter Estimation

给定观测数据，计算模型参数的后验概率；从而可以利用计算好的参数进行模型预测；


## 0.1 模型比较 Model Comparison

如果得到了多个模型，应该选择哪一个模型。

可以使用<b>贝叶斯模型平均</b>，而不仅仅使用某一个单独的模型。对得到的模型定义先验，然后使用模型的后验的平均结果来进行预测。

# 1. 非贝叶斯方法

## 1.1. 极大似然估计 Maximum Likelihood Estimation

比如有某个地区一批学生的身高数据，想知道该地区学生的生长发育情况。
假设学生的身高服从正态分布$X\sim \mathcal{N}(\mu,\sigma^2)$。
问题变成估计样本的均值$\mu$与方差$\sigma^2$。

$X$的似然函数为，
> $L(X;\mu,\sigma^2) = \prod_{n=1}^{N} \mathcal{N}(x_n; \mu,\sigma^2) $

防止似然函数下溢，计算对数似然，

> $l(X;\mu,\sigma^2) = \log L(X;\mu,\sigma^2) =
\log \prod_{n=1}^{N} \mathcal{N}(x_n; \mu,\sigma^2) $ 
> $ = \sum_{n=1}^{N} \log \mathcal{N}(x_n; \mu,\sigma^2)$

对数似然$l(X;\mu,\sigma^2)$关于$\mu$与$\sigma^2$的二阶导小于0，从而$l$存在极大值，为$l$关于$\mu$与$\sigma^2$的一阶导。

## 1.2. 正则化

## 1.3. EM算法

### 1.3.1. EM

如果模型中有与样本相关的隐含变量，则无法直接使用MLE。以K-Means方法为例，每个样本都属于$K$个类中的一个，但属于具体哪一个是未知的，从而也无法收集每个类的数据，对每个类计算聚类中心。

但是，如果隐含变量变成已知了，则可以使用MLE方法。

> + E-步：计算隐含变量（如每一个样本属于每个类的概率，属于哪个类）的后验概率，并计算完整数据关于模型参数（样本中心）的期望；

> + M-步：最大化模型参数的期望，得到模型参数；

以GMM为例，

> + E-步：计算隐含变量（每一个样本属于每个高斯分布的概率，softmax(概率密度值)），计算完整数据关于模型参数（每一个高斯分布的均值、方差）的似然的期望
$\mathbb{E}_{\mu,\sigma^2}[\ln p(X|\mu,\sigma^2) \ln p(\mu|\alpha) p(\sigma^2|\beta) ]$；

> + M-步：通过最大化期望得到$\mu$与$\sigma^2$的极大值；

实际上，在GMM中，$\mu$与$\sigma^2$是固定的参数，并没有设置先验分布，从而完整数据的期望是$\mathbb{E}_{\mu,\sigma^2}[\ln p(X|\mu,\sigma^2)]$。

### 1.3.2. MM 

根据需要寻找目标函数的最大值或者最小值被称为Minorize-Maxmization或者Majorize-Minimization。EM是MM的一个特例。

以寻找最大值为例，如果目标函数$f(\theta)$不可微，无法直接求极值。使用Minorize-Maxmization，寻找<b>替代函数</b>$g$，在第$m$步$g(\theta|\theta^{m})$有：

> + $\forall \theta, g(\theta|\theta^{m}) <= f(\theta)$
> + $g(\theta^{m}|\theta^{m}) = f(\theta^{m})$

则，

> + $g(\theta^{m+1}|\theta^{m}) = \arg\max_{\theta} g(\theta|\theta^{m}) $

因为，$f(\theta^{m+1}) >= g(\theta^{m+1}|\theta^{m}) >= g(\theta^{m}|\theta^{m}) = f(\theta^{m})$

### 1.3.3. ELBO Evidence Lower BOund

对于包含隐含变量$Z$的数据集$X$，根据概率公式

> $p(X, Z|\theta) = {p(Z|X, \theta)} p(X|\theta) $ 

$X$的对数似然为，
> $\ln p(X|\theta) = \ln \left ( \frac {p(X, Z|\theta)}{p(Z|X, \theta)} \right ) $

引入替代函数$q(Z)$，$\sum_{Z}q(Z) = 1$，

> $\ln p(X|\theta) = \left(\sum_{Z}q(Z)\right)
\ln p(X|\theta)$

> $ = \sum_{Z}q(Z) \ln \left (
 \frac{p(X, Z|\theta)}{p(Z|X, \theta)} \right ) $

> $ = \sum_{Z}q(Z) \ln \left (
 \frac{p(X, Z|\theta) / q(Z)}
 {p(Z|X, \theta) / q(Z)} \right )
$

> $ = \sum_{Z}q(Z) \left (
\ln \frac{p(X, Z|\theta)}{q(Z)} - 
\ln \frac{p(Z|X, \theta) }{q(Z)}
\right)$

> $ = \sum_{Z}q(Z) \ln \frac{p(X, Z|\theta)}{q(Z)} - 
\sum_{Z}q(Z) \ln \frac{p(Z|X, \theta) }{q(Z)}
$

> $ = \underbrace{ 
\sum_{Z}q(Z) \ln \frac{p(X, Z|\theta)}{q(Z)}}
_{\mathcal{L}(q,\theta)} + 
\underbrace{
\sum_{Z}q(Z) \ln \frac{q(Z)}{p(Z|X, \theta)}}
_{\mathbb{KL}(q(Z)||p(Z|X,\theta))}
$

> $ = \mathcal{L}(q,\theta) + 
\mathbb{KL}(q(Z)||p(Z|X,\theta))
$

> $ >= \mathcal{L}(q,\theta)$

所以，每次迭代可通过最大化$\mathcal{L}(q,\theta)$来最大化$\ln p(X|\theta)$。此时$\mathbb{KL}(q(Z)||p(Z|X,\theta))$取得最小值，即$q(Z) = p(Z|X,\theta))$。

选取$q(Z) = p(Z|X,\theta^{old}))$，

> $\mathcal{L}(q,\theta) = \sum_{Z} p(Z|X,\theta^{old}))
\ln \frac{p(X, Z|\theta)}{p(Z|X,\theta^{old}))}$

> $ = \underbrace{
\sum_{Z} p(Z|X,\theta^{old})) \ln p(X, Z|\theta)}_
{\mathcal{Q}(\theta, \theta^{old})} - 
\underbrace{
\sum_{Z} p(Z|X,\theta^{old})) p(Z|X,\theta^{old}))}_
{const}
$

> $ = \mathcal{Q}(\theta, \theta^{old}) + const $

然后最大化$\mathcal{Q}(\theta, \theta^{old})$，即完整数据的对数似然$\ln p(X, Z|\theta)$的期望。


# 2. 贝叶斯推断方法

## 2.1. 最大后验估计 Maximum A Posterior

MLE认为数据服从固定的参数决定的分布，贝叶斯学派认为参数是随机变量，本身也服从一定的分布。

将参数的先验分布考虑进来。假设

> $\mu \sim \mathcal{N}(0,1)$，$\sigma^2 \sim \mathcal{N}(0,1)$

由于 $posterior \propto likelihood \times prior$，从而

> $\mu^{*}, {\sigma}^{*} = 
\arg\max_{\mu, \sigma} 
\{\prod_{n=1}^{N} \mathcal{N}(x_n; \mu,\sigma^2)\} 
\mathcal{N}(\mu; 0,1) 
\mathcal{N}(\sigma^2; 0,1)$

## 2.2. 抽样方法 Sampling Methods

对于包含隐变量的模型参数估计，隐变量的后验概率$p(Z|X, \theta)$估计非常关键。

如果（1）隐变量的维度过高；（2）或者隐变量后验概率的期望非常难以计算；则不能直接使用EM算法来精确计算模型的参数。需要采用近似推断的方式。包括：（1）采用随机近似的<b>抽样方法</b>；（2）采用确定近似的<b>变分推断</b>方法。


## 2.3. 变分推断 Variational Inference

主要目的：用易计算的近似分布替代不易计算的参数后验概率分布。

主要问题：选择易计算的近似分布；尽可能替代原始的参数后验概率分布（KL散度）。


### 2.3.1. Intractability

（1）假设某个概率分布参数$\boldsymbol{\theta}$有500个，每一个有10,000个离散值，则需要计算500\*10,000个数值才能得到该概率分布的积分；

（2）常常是配分函数难以计算；

> $p(z|x) = \frac{p(z,x)}{p(x)} = 
\frac{p(x|z)p(z)}{p(x)}$

$p(x|z)$以及$p(z)$常常是用户自己选择的，易于计算。需要求解$p(x)$，并要求$\sum_{z}p(z|x) = 1$；

> $p(x) = \sum_{z}p(z|x)p(z)$

假设$z$有$M$维，每一个维度有$K$个状态，则需要遍历$K^{M}$个状态，才能计算得到$p(x)$。例如掷$M$个硬币，需要统计$2^{M}$个状态。

### 2.3.2 I-projection & M-projection

(1) <b>I-projection</b>

> $\mathbb{KL}(q||p) = \sum_{x}q(x)\ln \frac{q(x)}{p(x)}$

如果$p(x) = 0$，$q(x)$必须等于0，否则$\mathbb{KL}(q||p)$变成无穷大。
因此，I-projection是<b>zero-forcing</b>的，$q(x)$会低估(under-estimate)$p(x)$。

(2) <b>M-projection</b>

> $\mathbb{KL}(p||q) = \sum_{x}p(x)\ln \frac{p(x)}{q(x)}$

如果$p(x) > 0$，$q(x)$必须大于0，则M-projection是<b>zero-avoiding</b>的，$q(x)$会过估(over-estimate)$p(x)$。

I-projection会让$q(x)$趋近于一个单模态，用I-projection会让替代函数接近$p(x)$的局部分布。

### 2.3.3. 变分推断 Variational Inference

选择I-projection。

> $\ln p(X) = \mathcal{L}(q) + 
\mathbb{KL}(q(Z)||p(Z|X))$

> $ >= \mathcal{L}(q) = \sum_{Z}q(Z) 
\ln \frac{p(X,Z)}{q(Z)}$

与EM略有不同的是，认为模型参数$\theta$也是随机变量，从而可以与隐含变量$Z$一起计算。

如果隐含变量的后验概率分布$p(Z|X)$难以计算，选用易于计算的替代函数$q(Z)$。

### 2.3.4. 平均场理论 Mean Field Theory